In [2]:
import os
import sys
import pandas as pd

csv = 'dataset/cleand.csv'
data = pd.read_csv(csv,index_col=0, encoding='latin-1')
data.head()


,SentimentText,Sentiment
0,first think another disney movie might good it...,1
1,put aside dr house repeat missed desperate hou...,0
2,big fan stephen king s work film made even gre...,1
3,watched horrid thing tv needless say one movie...,0
4,truly enjoyed film acting terrific plot jeff c...,1


In [3]:
from sklearn.cross_validation import train_test_split

SEED = 2000

x_train, x_validation, y_train, y_validation = train_test_split(data.SentimentText, data.Sentiment, test_size=.2, random_state=SEED)

C:\Users\it\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Next, we will use Keras Tokenizer to split each word in a sentence. Then, in order to get a sequential representation of each row we use texts_to_sequences method.

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

C:\Users\it\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)

974

# We can say that the maximum lenght to be 1000

In [6]:
x_train_seq = pad_sequences(sequences, maxlen=1000)
x_train_seq[:5]

array([[   0,    0,    0, ..., 1265,   16,    6],
       [   0,    0,    0, ..., 1468,  213,  237],
       [   0,    0,    0, ...,   58,  100,    9],
       [   0,    0,    0, ...,   43,  664,  141],
       [   0,    0,    0, ...,    5,    1,  169]])

# After checking, we can see that all the data transformed to have the same length of 1000.

We do the same thing to the validation set.

In [7]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=1000)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from time import time

acc = []
times = []

model_cnn = Sequential()

e = Embedding(100000, 100, input_length=1000)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

t0 = time()
model_cnn.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
score,accu = model_cnn.evaluate(x_val_seq, y_validation, verbose = 2, batch_size = 32)
tv_time = time()-t0

acc.append(accu*100)
times.append(tv_time*0.0166667)

print("score: %.2f" % (score))
print("acc: %.2f" % (accu))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
 - 376s - loss: 0.3832 - acc: 0.8166 - val_loss: 0.2775 - val_acc: 0.8832
Epoch 2/5
 - 351s - loss: 0.1296 - acc: 0.9528 - val_loss: 0.3038 - val_acc: 0.8828
Epoch 3/5
 - 352s - loss: 0.0161 - acc: 0.9966 - val_loss: 0.3731 - val_acc: 0.8836
Epoch 4/5
 - 352s - loss: 0.0012 - acc: 1.0000 - val_loss: 0.4015 - val_acc: 0.8894
Epoch 5/5
 - 351s - loss: 2.3939e-04 - acc: 1.0000 - val_loss: 0.4156 - val_acc: 0.8890
score: 0.42
acc: 0.89


# Long Short Term Memory
Let's now try another model LSTM and compare it with the previous CNN model. We will use a single LSTM layer preceded by an embedding layer with 100000 as max feature and 128 dimension of each word in a sequence, then followed with a dense layer with softmax function.

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [10]:
#df = pd.read_csv('dataset/cleand.csv',delimiter=',',encoding='latin-1')
#df.head()
data = pd.read_csv('dataset/cleand.csv')
# Keeping only the neccessary columns
data = data[['SentimentText','Sentiment']]

In [11]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)

In [12]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 741, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 741, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(16750, 741) (16750, 2)
(8250, 741) (8250, 2)


In [14]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
 - 1310s - loss: 0.4644 - acc: 0.7781
Epoch 2/5
 - 1206s - loss: 0.3577 - acc: 0.8490
Epoch 3/5
 - 1652s - loss: 0.3075 - acc: 0.8749
Epoch 4/5
 - 2328s - loss: 0.2801 - acc: 0.8869
Epoch 5/5
 - 2927s - loss: 0.2551 - acc: 0.8996


In [15]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.37
acc: 0.85
